In [1]:
%cd ~/cdv
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import jax.numpy as jnp
import jax
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import rho_plus as rp

is_dark = False
theme, cs = rp.mpl_setup(is_dark)
rp.plotly_setup(is_dark)

/home/nmiklaucic/cdv


/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from pathlib import Path
from facet.data.databatch import CrystalGraphs
from facet.layers import Context
import pyrallis
from facet.config import MainConfig
import orbax.checkpoint as ocp

from facet.training_state import TrainingRun
from facet.checkpointing import best_ckpt

from pathlib import Path
import pyrallis
from facet.config import MainConfig
import orbax.checkpoint as ocp

from facet.training_state import TrainingRun
from facet.checkpointing import best_ckpt

pretrained = True

if pretrained:
    run_dir = Path('logs') / 'enb-178'
    # run_dir = Path('logs') / '10-04-13-00_430'
    conf_file = run_dir / 'config.toml'
else:
    conf_file = 'configs/small.toml'

with open(conf_file) as f:
    config = pyrallis.cfgparsing.load(MainConfig, f)
    config.batch_size = 32

if not pretrained:    
    config.model.resid_init = 'ones'

# config.model.self_connection.mlp.dropout = 0.0
model = config.build_regressor()

from facet.data.dataset import load_file

config.data.dataset_name = 'mptrj'
cgs = []
for i in range(1):
    cgs.append(load_file(config, group_num=0, file_num=i))

cg: CrystalGraphs = sum(cgs[1:], start=cgs[0])

params = model.init(jax.random.key(29205), cg=cg, ctx=Context(training=True))

mod = model.bind(params)
param_size = sum(x.size for x in jax.tree.leaves(params))
param_size

In [16]:
from facet.data.dataset import load_file, dataloader

num_batches, dl = dataloader(config, 'train')
cgs = next(dl)

In [21]:
fn = lambda cg: model.apply(params, cg=cg, ctx=Context(training=False))

jax.pmap(fn)(cgs)


Array([[[ -7.391423 ],
        [ -7.739697 ],
        [ -7.9990926],
        [ -5.105004 ],
        [ -6.5087214],
        [ -3.4455237],
        [ -8.521961 ],
        [ -6.1716256],
        [ -4.794539 ],
        [ -7.5928893],
        [ -6.299146 ],
        [ -7.2680087],
        [ -6.840678 ],
        [ -4.9852934],
        [ -6.323713 ],
        [ -6.1229486],
        [ -9.194131 ],
        [ -8.356192 ],
        [ -5.106946 ],
        [ -2.7450523],
        [ -6.113339 ],
        [ -6.0432096],
        [ -5.6926737],
        [ -4.3806043],
        [ -2.8805754],
        [ -3.3481693],
        [ -2.2863467],
        [ -7.221292 ],
        [ -6.724867 ],
        [ -9.524621 ],
        [ -3.638401 ],
        [ -6.190838 ]],

       [[ -7.6177087],
        [ -6.4045467],
        [ -7.2617135],
        [ -6.246022 ],
        [ -8.064104 ],
        [ -6.1022415],
        [ -8.761226 ],
        [ -7.584927 ],
        [ -6.569234 ],
        [ -8.125051 ],
        [ -7.932147 ],
        

In [3]:
from facet.utils import debug_structure


debug_structure(params);

arg0 >>> params
├── edge_embedding
│   ├── basis >>> freq
│   │   └── f32[8]
│   └── rmax
│       └── f32[1]
├── head >>> Dense_0 >>> kernel
│   └── f32[64, 1]
├── mace
│   ├── layer_0
│   │   ├── interaction
│   │   │   ├── SimpleInteraction_0
│   │   │   │   ├── SevenNetConv_0
│   │   │   │   │   └── {...}
│   │   │   │   └── linear_outro
│   │   │   │       └── {...}
│   │   │   └── resid_adapter >>> w[0,0] 64x0e,128x0e
│   │   │       └── f32[64, 128]
│   │   ├── resid_ln >>> ln_scale
│   │   │   └── f32[224]
│   │   └── self_connection
│   │       ├── mlp
│   │       │   ├── Dense_0
│   │       │   │   └── {...}
│   │       │   └── Dense_1
│   │       │       └── {...}
│   │       ├── proj_down
│   │       │   ├── w[0,0] 192x0e,128x0e
│   │       │   │   └── f32[192, 128]
│   │       │   ├── w[1,1] 64x1e,64x1e
│   │       │   │   └── f32[64, 64]
│   │       │   └── w[2,2] 64x2e,32x2e
│   │       │       └── f32[64, 32]
│   │       └── proj_up
│   │           ├── w[0,0] 128x0e,64x0e
│   │           │   └── f32[128, 64]
│   │           ├── w[1,1] 64x1e,64x1e
│   │           │   └── f32[64, 64]
│   │           └── w[2,2] 32x2e,64x2e
│   │               └── f32[32, 64]
│   ├── layer_1
│   │   ├── interaction
│   │   │   ├── SimpleInteraction_0
│   │   │   │   ├── SevenNetConv_0
│   │   │   │   │   └── {...}
│   │   │   │   └── linear_outro
│   │   │   │       └── {...}
│   │   │   └── resid_adapter
│   │   │       ├── w[0,0] 128x0e,128x0e
│   │   │       │   └── f32[128, 128]
│   │   │       ├── w[1,1] 64x1e,64x1e
│   │   │       │   └── f32[64, 64]
│   │   │       └── w[2,2] 32x2e,32x2e
│   │   │           └── f32[32, 32]
│   │   ├── resid_ln >>> ln_scale
│   │   │   └── f32[224]
│   │   └── self_connection
│   │       ├── mlp
│   │       │   ├── Dense_0
│   │       │   │   └── {...}
│   │       │   └── Dense_1
│   │       │       └── {...}
│   │       ├── proj_down
│   │       │   ├── w[0,0] 192x0e,128x0e
│   │       │   │   └── f32[192, 128]
│   │       │   ├── w[1,1] 64x1e,64x1e
│   │       │   │   └── f32[64, 64]
│   │       │   └── w[2,2] 64x2e,32x2e
│   │       │       └── f32[64, 32]
│   │       └── proj_up
│   │           ├── w[0,0] 128x0e,64x0e
│   │           │   └── f32[128, 64]
│   │           ├── w[1,1] 64x1e,64x1e
│   │           │   └── f32[64, 64]
│   │           └── w[2,2] 32x2e,64x2e
│   │               └── f32[32, 64]
│   └── layer_2
│       ├── interaction
│       │   ├── SimpleInteraction_0
│       │   │   ├── SevenNetConv_0
│       │   │   │   └── {...}
│       │   │   └── linear_outro
│       │   │       └── {...}
│       │   └── resid_adapter >>> w[0,0] 128x0e,64x0e
│       │       └── f32[128, 64]
│       ├── resid_ln >>> ln_scale
│       │   └── f32[64]
│       └── self_connection >>> LazyInMLP_0
│           ├── Dense_0 >>> kernel
│           │   └── f32[64, 64]
│           ├── Dense_1 >>> kernel
│           │   └── f32[64, 64]
│           └── Dense_2 >>> kernel
│               └── f32[64, 64]
├── node_embedding >>> embed >>> embedding
│   └── f32[90, 64]
└── rescale
    ├── global_scale
    │   └── f32[1]
    ├── global_shift
    │   └── f32[1]
    ├── scale
    │   └── f32[90]
    └── shift
        └── f32[90]

In [10]:
mod.mace.layers[0].interaction


ResidualInteraction(
    # attributes
    irreps_out = (128x0e, 64x1e, 32x2e)
    interaction = SimpleInteraction(
        # attributes
        irreps_out = None
        conv = SevenNetConv(
            # attributes
            irreps_out = None
            avg_num_neighbors = None
            max_ell = 2
            radial_weight = LazyInMLP(
                # attributes
                inner_dims = ()
                out_dim = None
                residual = False
                inner_act = silu
                final_act = Identity()
                dropout_rate = 0.0
                kernel_init = init
                bias_init = init
                normalization = 'Identity'
                use_bias = False
                # children
                final_act = Identity()
            )
            radial_power = 0.7
            # children
            radial_weight = LazyInMLP(
                # attributes
                inner_dims = ()
                out_dim = None
            

In [6]:
config


MainConfig(
    batch_size=32,
    stack_size=1,
    num_epochs=60,
    restart_from=None,
    checkpoint_params=None,
    precision='f32',
    debug_mode=False,
    do_profile=False,
    display='progress',
    data=DataConfig(
        dataset_name='mptrj',
        raw_data_folder=PosixPath('data'),
        data_folder=PosixPath('precomputed'),
        shuffle_seed=29205,
        train_split=30,
        test_split=3,
        valid_split=3,
        batches_per_group=0,
        batch_n_nodes=None,
        k=48,
        batch_n_graphs=None
    ),
    cli=CLIConfig(verbosity=<LoggingLevel.info: 20>, show_progress=True),
    device=DeviceConfig(device='gpu', max_gpus=0, gpu_ids=[]),
    log=LogConfig(
        log_dir=PosixPath('logs'),
        exp_name=None,
        logs_per_epoch=32,
        epochs_per_ckpt=2,
        epochs_per_valid=0.5,
        tags=[],
        log_params=[
            'edge_embedding/basis/mu',
            'edge_embedding/basis/sigma',
            'edge_embedding/bas

In [4]:
def out(params):
    return model.apply(params, cg=cg, ctx=Context(False), rngs=jax.random.key(123)).reshape(-1)

out(params)


Array([ -8.171735 ,  -7.4753838,  -5.069386 ,  -4.1679845,  -5.9924464,
        -8.374617 ,  -6.979674 ,  -8.293659 ,  -5.7668777,  -5.562835 ,
        -4.2132077,  -8.313215 ,  -7.3745565,  -7.8405905,  -4.592948 ,
        -6.849234 ,  -7.4770746,  -8.190898 ,  -5.753566 ,  -6.5061274,
       -10.138165 ,  -5.160405 ,  -8.243992 ,  -5.058351 ,  -7.2617745,
        -3.936257 ,  -8.234032 ,  -6.912834 , -11.538978 ,  -7.7584867,
        -6.9060397,  -6.1870656], dtype=float32)

In [15]:
from copy import deepcopy


new_params = deepcopy(params)
layer = new_params['params']['mace']['layer_1']

left = layer['interaction']['SimpleInteraction_0']['linear_outro']
right = layer['self_connection']['proj_up']
debug_structure(left=left, right=right);
for k1, k2 in zip(sorted(left), sorted(right)):    
    if left[k1].shape == right[k2].shape:
        print(k1, k2)
        left[k1] = left[k1] @ right[k2]
        right[k2] = jnp.eye(*right[k2].shape)



left
├── w[0,0] 224x0e,128x0e
│   └── f32[224, 128]
├── w[1,1] 384x1e,64x1e
│   └── f32[384, 64]
└── w[2,2] 352x2e,32x2e
    └── f32[352, 32]

right
├── w[0,0] 128x0e,64x0e
│   └── f32[128, 64]
├── w[1,1] 64x1e,64x1e
│   └── f32[64, 64]
└── w[2,2] 32x2e,64x2e
    └── f32[32, 64]

In [14]:
out(new_params)


Array([ -8.171735 ,  -7.4753838,  -5.069385 ,  -4.1679845,  -5.9924464,
        -8.374617 ,  -6.979674 ,  -8.293659 ,  -5.7668777,  -5.562835 ,
        -4.2132077,  -8.313215 ,  -7.3745565,  -7.8405905,  -4.592948 ,
        -6.849234 ,  -7.477074 ,  -8.190898 ,  -5.753566 ,  -6.5061274,
       -10.138165 ,  -5.160405 ,  -8.243992 ,  -5.058351 ,  -7.2617745,
        -3.936257 ,  -8.234032 ,  -6.912834 , -11.538978 ,  -7.7584867,
        -6.9060397,  -6.1870656], dtype=float32)